In [26]:


import tensorflow as tf
from tensorflow import keras
from keras import layers


tf.config.run_functions_eagerly(True)


(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
x_val, y_val = x_train[-5000:], y_train[-5000:]
x_train, y_train = x_train[:-5000], y_train[:-5000]


In [27]:

class AddGaussianNoise(layers.Layer):
    def __init__(self, stddev=0.1):
        super().__init__()
        self.stddev = stddev
    def call(self, inputs, training=False):
        if training:
            return inputs + tf.random.normal(tf.shape(inputs), stddev=self.stddev)
        return inputs


inputs = keras.Input(shape=(28, 28, 1))
x = layers.Flatten()(inputs)
x = AddGaussianNoise(0.1)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(10)(x)
outputs = layers.Activation("softmax")(x)
model = keras.Model(inputs, outputs)




In [28]:
learning_rate = 0.01
momentum = 0.9
velocities = [tf.Variable(tf.zeros_like(var)) for var in model.trainable_variables]

loss_fn = keras.losses.SparseCategoricalCrossentropy()
train_acc = keras.metrics.SparseCategoricalAccuracy()
val_acc = keras.metrics.SparseCategoricalAccuracy()

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(64)
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(64)


In [29]:

for epoch in range(5):
    print(f"\nEpoch {epoch+1}/5")
    for x_batch, y_batch in train_ds:
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)

        for v, g, param in zip(velocities, grads, model.trainable_variables):
            v.assign(momentum * v - learning_rate * g)
            param.assign_add(v)

        train_acc.update_state(y_batch, logits)

    print(f"Train Accuracy: {train_acc.result().numpy():.4f}")
    train_acc.reset_state()

    for x_batch, y_batch in val_ds:
        val_preds = model(x_batch, training=False)
        val_acc.update_state(y_batch, val_preds)
    print(f"Val Accuracy: {val_acc.result().numpy():.4f}")
    val_acc.reset_state()




Epoch 1/5
Train Accuracy: 0.7807
Val Accuracy: 0.8394

Epoch 2/5
Train Accuracy: 0.8427
Val Accuracy: 0.8482

Epoch 3/5
Train Accuracy: 0.8572
Val Accuracy: 0.8674

Epoch 4/5
Train Accuracy: 0.8631
Val Accuracy: 0.8732

Epoch 5/5
Train Accuracy: 0.8716
Val Accuracy: 0.8654


In [30]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)
test_acc = keras.metrics.SparseCategoricalAccuracy()
for x_batch, y_batch in test_ds:
    preds = model(x_batch, training=False)
    test_acc.update_state(y_batch, preds)
print(f"\nTest Accuracy: {test_acc.result().numpy():.4f}")


Test Accuracy: 0.8593
